## Часть 1
Напишите скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227). Затем нужно перейти на сайты региональных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название региона
- название ТИК
- номер УИК
- 20 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [ ]:
import lxml.html
import requests
import tqdm
from multiprocessing import Pool
import csv

def getSubjectInfo(path):
    response = requests.get(path)
    tree = lxml.html.fromstring(response.text)
    subj_locations = tree.xpath('//a[contains(@style, "text-decoration: none")]/@href')
    subj_names = tree.xpath('//a[contains(@style, "text-decoration: none")]/text()')
    result = []
    for i in range(len(subj_locations)):
        result.append([subj_locations[i], subj_names[i]])
    return result


def parseSubject(info):
    response = requests.get(info[0])
    tree = lxml.html.fromstring(response.text)
    tik_locations = tree.xpath('//option/@value')
    tik_names = tree.xpath('//option/text()')[1:]

    result = []
    if len(tik_locations) == 0:
        yik_location = tree.xpath('//a/@href')
        response2 = requests.get(yik_location[4])
        tree2 = lxml.html.fromstring(response2.text)
        table_part = list(
            map(int, tree2.xpath('//table[contains(@style, "width:100%;overflow:scroll")]//td//nobr//b//text()')))
        yik_numbers = tree2.xpath('//table[contains(@style, "width:100%;overflow:scroll")]//td//nobr/text()')
        temp = [len(yik_numbers), info[1], 'None'] + yik_numbers + table_part
        result.append(temp)
        return result


    for k, link in enumerate(tik_locations):
        response2 = requests.get(link)
        tree2 = lxml.html.fromstring(response2.text)
        yik_locations = tree2.xpath('//a/@href')

        response3 = requests.get(yik_locations[5])
        tree3 = lxml.html.fromstring(response3.text)
        table_part = list(
            map(int, tree3.xpath('//table[contains(@style, "width:100%;overflow:scroll")]//td//nobr//b//text()')))
        yik_numbers = tree3.xpath('//table[contains(@style, "width:100%;overflow:scroll")]//td//nobr/text()')

        temp = [len(yik_numbers), info[1], tik_names[k]] + yik_numbers + table_part
        result.append(temp)

    return result


def csv_writer(data, path):
    data = prepare_data(data)
    with open(path, "a", newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        for line in data:
            writer.writerow(line)


def prepare_data(data):
    result = []
    for region in data:
        temp = [region[1:3] for x in range(region[0])]
        for i in range(3, len(region)):
            temp[(i - 3) % region[0]].append(region[i])
        result = result + temp
    return result

entrance = 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn' \
           '=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'
path = '/home/alex/PycharmProjects/alms-classification-2018-88a3f24a5b0e/tasks/data.csv'

subj_locations_and_names = getSubjectInfo(entrance)
res = parseSubject(subj_locations_and_names[0])

pool = Pool(processes=4)
for iter in tqdm.tqdm(pool.imap(parseSubject, subj_locations_and_names), total=len(subj_locations_and_names)):
    csv_writer(iter, path)


### Часть 2
Используя скаченные данные и `Pandas` (или `PySpark`) посчитайте:
- явку (%) по всем регионам, результат отсортировать по убыванию
- выберите произвольного кандидата и найдите тот избиратльный участок, на котором он получил наибольший результат  (%, учитывать участки на которых проголосовало больше 300 человек)
- найдите регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна
- посчитайте дисперсию по явке для каждого региона (учитывать УИК)
- для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [1]:
import pandas as pd

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('data.csv')

# явку (%) по всем регионам, результат отсортировать по убыванию #

In [3]:
a = data.groupby('Регион')['9', '10', '12', '1'].sum()
((a['9']+a['10']+a['12'])/a['1']).sort_values(ascending=False)

Регион
Территория за пределами РФ             0.980182
Республика Тыва                        0.936254
Ямало-Ненецкий автономный округ        0.918721
Кабардино-Балкарская Республика        0.916983
Чеченская Республика                   0.915114
Республика Северная Осетия - Алания    0.899370
Республика Дагестан                    0.874450
Карачаево-Черкесская Республика        0.873438
Кемеровская область                    0.830756
Чукотский автономный округ             0.822605
Республика Ингушетия                   0.819594
Брянская область                       0.796598
Тюменская область                      0.788830
Республика Мордовия                    0.778365
Краснодарский край                     0.778347
Республика Татарстан (Татарстан)       0.773653
Чувашская Республика - Чувашия         0.762107
Республика Башкортостан                0.754212
Республика Бурятия                     0.751913
Республика Адыгея (Адыгея)             0.743060
Ставропольский край              

In [5]:
data['Sum'] = data['9']+data['10']+data['12']

# выберите произвольного кандидата и найдите тот избиратльный участок, на котором он получил наибольший результат (%, учитывать участки на которых проголосовало больше 300 человек)

In [6]:
# 14:'Грудинин Павел Николаевич' 
t = data[data['Sum']>300]
t.iloc[[(t['14']/t['Sum']).idxmax()]]

,Регион,ТИК,УИК,1,2,3,4,5,6,7,...,12,13,14,15,16,17,18,19,20,Sum
78587,Свердловская область,"29 Екатеринбург, Чкаловская",УИК №1784,1772,1750,0,1046,20,684,20,...,0,9,91,46,866,14,3,14,15,1065


# найдите регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [134]:
t = data.groupby('Регион')['Sum'].max()- data.groupby('Регион')['Sum'].min()
t.idxmax()

'Территория за пределами РФ'

# посчитайте дисперсию по явке для каждого региона (учитывать УИК)

In [138]:
data.groupby('Регион')['Sum'].var()

Регион
Алтайский край                              2.383965e+05
Амурская область                            1.897785e+05
Архангельская область                       2.713373e+05
Астраханская область                        2.213151e+05
Белгородская область                        1.817451e+05
Брянская область                            3.502155e+05
Владимирская область                        2.318411e+05
Волгоградская область                       2.877098e+05
Вологодская область                         3.217944e+05
Воронежская область                         1.763268e+05
Город Байконур (Республика Казахстан)       3.275048e+04
Еврейская автономная область                1.428222e+05
Забайкальский край                          2.191117e+05
Ивановская область                          1.051740e+05
Иркутская область                           1.962033e+05
Кабардино-Балкарская Республика             3.772411e+05
Калининградская область                     2.533419e+05
Калужская область       

# для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [151]:
candidates = data[['12','13','14','15','16','17','18','19','20']]
percentage = candidates.div(data['Sum']*0.01, axis=0).round()
percentage.apply(pd.Series.value_counts).fillna(0).astype(int)


,12,13,14,15,16,17,18,19,20
0.0,97689,45203,1181,4014,1,27992,43531,49976,51358
1.0,3,47849,2163,4739,0,36346,48776,39981,30773
2.0,0,3734,3111,4850,0,18563,3960,6571,9086
3.0,1,526,3387,6669,0,7812,768,843,3677
4.0,0,179,3662,10269,1,3562,253,178,1580
5.0,0,89,3856,13261,0,1610,131,51,708
6.0,0,35,4007,13742,1,758,82,26,299
7.0,0,20,4548,11806,0,450,43,22,117
8.0,1,22,5273,8843,1,227,37,17,49
9.0,0,10,6079,6248,0,118,30,3,13
